<a href="https://colab.research.google.com/github/Jorisvdv/Food_Kaggle_UvA/blob/master/Food_Challenge_Bagged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, model_from_json, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D, Activation, Flatten, BatchNormalization
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from tensorflow.keras.losses import categorical_crossentropy
import numpy as np
from sklearn.utils import class_weight
import os.path
import fnmatch
import itertools
import functools
from math import floor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

#Different Models
from tensorflow.keras.applications.inception_v3 import InceptionV3
# Additions for model Antonio
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import regularizers
import json

# Added ResNet model
from tensorflow.keras.applications.resnet50 import ResNet50



# Download Files

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)
!mkdir ~/.kaggle
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Food\ Kaggle/kaggle.json ~/.kaggle/kaggle.json

# Make output file for models
!mkdir competitions/food-recognition-challenge/models

from google.colab import files
# !ls
# print("Upload kaggle api credentials json")
# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
mkdir: cannot create directory ‘/root/.kaggle’: File exists
mkdir: cannot create directory ‘competitions/food-recognition-challenge/models’: File exists


In [3]:
# # Set up kaggle, for some reason this does not download all files.
!pip install --upgrade --no-deps --force-reinstall  kaggle
# !mkdir ~/.kaggle
# !cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json
!kaggle config set -n path -v /content

!kaggle competitions download food-recognition-challenge
# Unzip data
!unzip -q -n competitions/food-recognition-challenge/food-recognition-challenge.zip -d competitions/food-recognition-challenge/
!ls competitions/food-recognition-challenge/
print(len([name for name in os.listdir('competitions/food-recognition-challenge/train_set/train_set')]))
print(len([name for name in os.listdir('competitions/food-recognition-challenge/test_set/test_set')]))

Processing /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674/kaggle-1.5.6-cp36-none-any.whl
  Found existing installation: kaggle 1.5.6
    Uninstalling kaggle-1.5.6:
      Successfully uninstalled kaggle-1.5.6
- path is now set to: /content
food-recognition-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)
food-recognition-challenge.zip	sample.csv	  train_set
i3_final.hdf5			submission.csv	  vgg_checkpoint.hdf5
i3_top_checkpoint.hdf5		test_set	  vgg_final.hdf5
models				train_labels.csv
30612
7653


# General Settings


In [4]:
# Settings
img_width, img_height = 224, 224
i3_img_width, i3_img_height = 299, 299
data = pd.read_csv('competitions/food-recognition-challenge/train_labels.csv',dtype=str)
display(data.head())
train_data_dir = 'competitions/food-recognition-challenge/train_set/train_set/'
test_data_dir = 'competitions/food-recognition-challenge/test_set/test_set/'

checkpoint_vgg= 'competitions/food-recognition-challenge/models/vgg_checkpoint.hdf5'
final_vgg= 'competitions/food-recognition-challenge/models/vgg_final.hdf5'
json_vgg = "competitions/food-recognition-challenge/models/vgg_model.json"
weights_vgg = "competitions/food-recognition-challenge/models/vgg_model_weights.h5"

checkpoint_top_i3 = 'competitions/food-recognition-challenge/models/i3_top_checkpoint.hdf5'
checkpoint_full_i3 = 'competitions/food-recognition-challenge/models/i3_full_checkpoint.hdf5'
final_i3= 'competitions/food-recognition-challenge/models/i3_final.hdf5'
json_i3 = "competitions/food-recognition-challenge/models/i3_model.json"
weights_i3 = "competitions/food-recognition-challenge/models/i3_model_weights.h5"

checkpoint_top_resnet = 'competitions/food-recognition-challenge/models/resnet_top_checkpoint.hdf5'
checkpoint_full_resnet = 'competitions/food-recognition-challenge/models/resnet_full_checkpoint.hdf5'
final_resnet= 'competitions/food-recognition-challenge/models/resnet_final.hdf5'
json_resnet = "competitions/food-recognition-challenge/models/resnet_model.json"
weights_resnet = "competitions/food-recognition-challenge/models/resnet_model_weights.h5"

submission_file = "competitions/food-recognition-challenge/submission.csv"

epochs_top = 1
epochs = 1
batch_size = 64
patience_num = 3

# Load in model files
vgg_model_file = False #"../input/model-antonio-v4/model_more_fully.h5"
vgg_model_json_file = False #"../input/model-antonio-v4/model_more_fully.json"
i3_model_file = False #"../input/inception-food-model/trained_model.hdf5"
i3_model_json_file = False
resnet_model_file = False #"../input/inception-food-model/trained_model.hdf5"
resnet_model_json_file = False



#Train-test split
data_h, holdoutSet = train_test_split(data, test_size=0.1, random_state = 21)
#Train-test split
trainingSet, validationSet = train_test_split(data_h, test_size=0.2, random_state = 16)


# Test flow_from_dataframe
sample_csv = pd.read_csv('competitions/food-recognition-challenge/sample.csv',dtype=str)


,img_name,label
0,train_1.jpg,21
1,train_2.jpg,29
2,train_3.jpg,17
3,train_4.jpg,21
4,train_5.jpg,50


# DataLoaders

In [5]:
# Create class weights

class_w = class_weight.compute_class_weight('balanced', np.unique(trainingSet["label"]), trainingSet["label"])

#VGG Loaders
def preprocess_input_vgg(x):
    """Wrapper around keras.applications.vgg16.preprocess_input()
    to make it compatible for use with keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument.
    
    Parameters
    ----------
    x : a numpy 3darray (a single image to be preprocessed)
    
    Note we cannot pass keras.applications.vgg16.preprocess_input()
    directly to to keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument because the former expects a
    4D tensor whereas the latter expects a 3D tensor. Hence the
    existence of this wrapper.
    
    Returns a numpy 3darray (the preprocessed image).
    
    """
    X = np.expand_dims(x, axis=0)
    X = preprocess_input(X)
    return X[0]

vgg_train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

vgg_train_generator = vgg_train_datagen.flow_from_dataframe(
    dataframe= trainingSet,
    x_col="img_name",
    y_col="label",
    directory=train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
    seed = 3)

vgg_validation_train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg)
vgg_validation_generator = vgg_validation_train_datagen.flow_from_dataframe(
    dataframe= validationSet,
    x_col="img_name",
    y_col="label",
    directory=train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
    seed = 4)

vgg_hold_generator = vgg_validation_train_datagen.flow_from_dataframe(
    dataframe= holdoutSet,
    x_col="img_name",
    y_col="label",
    directory=train_data_dir,
    target_size=(img_width, img_height),
    shuffle=True,
    class_mode="categorical",
    batch_size=batch_size)

vgg_test_generator = vgg_validation_train_datagen.flow_from_dataframe(
    dataframe= sample_csv,
    x_col="img_name",
    y_col="label",
    directory=test_data_dir,
    target_size=(img_width, img_height),
    shuffle=False,
    class_mode=None,
    batch_size=batch_size)

vgg_train_samples = len(vgg_train_generator.filenames)
vgg_validation_samples = len(vgg_validation_generator.filenames)

print("vgg_Train files: ", len(vgg_train_generator.filenames))
print("vgg_Validation files: ", len(vgg_validation_generator.filenames))
print("i3_Holdout files: ", len(vgg_hold_generator.filenames))
print("vgg_Test files: ", len(vgg_test_generator.filenames))

vgg_mapping_indices = vgg_train_generator.class_indices
print(vgg_mapping_indices)


# Inception V3
i3_train_datagen=ImageDataGenerator(rescale=1./255.,
                              rotation_range=40,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.2,
                              zoom_range=0.2,
                              horizontal_flip=True,
                              fill_mode='nearest')

i3_train_generator=i3_train_datagen.flow_from_dataframe(
    dataframe= trainingSet,
    directory=train_data_dir,
    x_col="img_name",
    y_col='label',
    batch_size=batch_size,
    seed=5,
    shuffle=True,
    class_mode="categorical",
    target_size=(i3_img_width, i3_img_height))

i3_validation_train_datagen=ImageDataGenerator(rescale=1./255.)

i3_validation_generator=i3_validation_train_datagen.flow_from_dataframe(
    dataframe= validationSet,
    directory=train_data_dir,
    x_col="img_name",
    y_col='label',
    batch_size=batch_size,
    seed=6,
    shuffle=True,
    class_mode="categorical",
    target_size=(i3_img_width, i3_img_height))

i3_hold_generator = i3_validation_train_datagen.flow_from_dataframe(
    dataframe= holdoutSet,
    directory=train_data_dir,
    x_col="img_name",
    y_col="label",
    target_size=(i3_img_width, i3_img_height),
    shuffle=True,
    class_mode="categorical",
    batch_size=batch_size)

i3_test_generator = i3_validation_train_datagen.flow_from_dataframe(
    dataframe= sample_csv,
    x_col="img_name",
    y_col="label",
    directory=test_data_dir,
    target_size=(i3_img_width, i3_img_height),
    shuffle=False,
    class_mode=None,
    batch_size=batch_size)

# filenames = test_generator.filenames
# nb_samples = len(filenames)
# print(nb_samples)

i3_mapping_indices = i3_train_generator.class_indices

i3_train_samples = len(i3_train_generator.filenames)
i3_validation_samples = len(i3_validation_generator.filenames)

print("i3_Train files: ", len(i3_train_generator.filenames))
print("i3_Validation files: ", len(i3_validation_generator.filenames))
print("i3_Holdout files: ", len(i3_hold_generator.filenames))
print("i3_Test files: ", len(i3_test_generator.filenames))

# resnet
resnet_train_datagen=ImageDataGenerator(rescale=1./255.,
                              rotation_range=40,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.2,
                              zoom_range=0.2,
                              horizontal_flip=True,
                              fill_mode='nearest')

resnet_train_generator=resnet_train_datagen.flow_from_dataframe(
    dataframe= trainingSet,
    directory=train_data_dir,
    x_col="img_name",
    y_col='label',
    batch_size=batch_size,
    seed=7,
    shuffle=True,
    class_mode="categorical",
    target_size=(img_width, img_height))

resnet_validation_train_datagen=ImageDataGenerator(rescale=1./255.)

resnet_validation_generator=resnet_validation_train_datagen.flow_from_dataframe(
    dataframe= validationSet,
    directory=train_data_dir,
    x_col="img_name",
    y_col='label',
    batch_size=batch_size,
    seed=8,
    shuffle=True,
    class_mode="categorical",
    target_size=(img_width, img_height))

resnet_hold_generator = resnet_validation_train_datagen.flow_from_dataframe(
    dataframe= holdoutSet,
    directory=train_data_dir,
    x_col="img_name",
    y_col="label",
    target_size=(img_width, img_height),
    shuffle=True,
    seed=9,
    class_mode="categorical",
    batch_size=batch_size)

resnet_test_generator = resnet_validation_train_datagen.flow_from_dataframe(
    dataframe= sample_csv,
    x_col="img_name",
    y_col="label",
    directory=test_data_dir,
    target_size=(img_width, img_height),
    shuffle=False,
    class_mode=None,
    batch_size=batch_size)


resnet_train_samples = len(resnet_train_generator.filenames)
resnet_validation_samples = len(resnet_validation_generator.filenames)

print("Resnet_Train files: ", len(resnet_train_generator.filenames))
print("Resnet_Validation files: ", len(resnet_validation_generator.filenames))
print("Resnet_Holdout files: ", len(resnet_hold_generator.filenames))
print("Resnet_Test files: ", len(resnet_test_generator.filenames))

resnet_mapping_indices = resnet_train_generator.class_indices

if i3_mapping_indices == vgg_mapping_indices ==resnet_mapping_indices:
  print("Indices match")
else:
  print("Indices do not match")

Found 22040 validated image filenames belonging to 80 classes.
Found 5510 validated image filenames belonging to 80 classes.
Found 3062 validated image filenames belonging to 80 classes.
Found 7653 validated image filenames.
vgg_Train files:  22040
vgg_Validation files:  5510
i3_Holdout files:  3062
vgg_Test files:  7653
{'1': 0, '10': 1, '11': 2, '12': 3, '13': 4, '14': 5, '15': 6, '16': 7, '17': 8, '18': 9, '19': 10, '2': 11, '20': 12, '21': 13, '22': 14, '23': 15, '24': 16, '25': 17, '26': 18, '27': 19, '28': 20, '29': 21, '3': 22, '30': 23, '31': 24, '32': 25, '33': 26, '34': 27, '35': 28, '36': 29, '37': 30, '38': 31, '39': 32, '4': 33, '40': 34, '41': 35, '42': 36, '43': 37, '44': 38, '45': 39, '46': 40, '47': 41, '48': 42, '49': 43, '5': 44, '50': 45, '51': 46, '52': 47, '53': 48, '54': 49, '55': 50, '56': 51, '57': 52, '58': 53, '59': 54, '6': 55, '60': 56, '61': 57, '62': 58, '63': 59, '64': 60, '65': 61, '66': 62, '67': 63, '68': 64, '69': 65, '7': 66, '70': 67, '71': 68, '72

# Model VGG16

In [6]:
if os.path.isfile(final_vgg):
  vgg_model = load_model(final_vgg)
elif os.path.isfile(vgg_model_file):
  if os.path.isfile(vgg_model_json_file):
    with open(vgg_model_json_file,'r') as f:
      vgg_model = model_from_json(f.read())
      vgg_model.load_weights(vgg_model_file)
  else:
    vgg_model = load_model(vgg_model_file)
else:

    vgg16 = VGG16(weights='imagenet', include_top = True)
    vgg16.summary()
    x  = vgg16.get_layer('fc2').output
    # x = Flatten(name='flatten2')(x)
    x = Dense(4096, activation='relu', name='d1')(x)
    x = Dropout(0.1)(x)
    x = Dense(4096, activation='relu', name='d2')(x)
    prediction = Dense(80, activation='softmax', name='predictions')(x)

    vgg_model = Model(inputs=vgg16.input, outputs=prediction)
    vgg_model.summary()



# Freeze All Layers Except Bottleneck Layers for Fine-Tuning

for layer in vgg_model.layers:
    if layer.name in ['predictions', 'd1', 'd2', 'block4_conv3', 'block5_conv1', 'block5_conv2', 'block5_conv3']:
        continue
    layer.trainable = False


df = pd.DataFrame(([layer.name, layer.trainable] 
                  for layer in vgg_model.layers), columns=['layer', 'trainable'])

sgd = SGD(lr=1e-4, momentum=0.9, clipvalue = 0.5)

vgg_model.compile(optimizer=sgd, 
                loss='categorical_crossentropy', metrics=['accuracy'])

# serialize model to JSON
model_json = vgg_model.to_json()
with open(json_vgg, "w") as json_file:
    json_file.write(model_json)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


# Inception V3 Model

In [0]:
# Inception V3 model
base_i3_model = InceptionV3(weights='imagenet', include_top=False)

if os.path.isfile(final_i3):
  i3_model = load_model(final_i3)
elif os.path.isfile(i3_model_file):
  if os.path.isfile(i3_model_json_file):
    with open(i3_model_json_file,'r') as f:
      i3_model = model_from_json(f.read())
      i3_model.load_weights(i3_model_file)
  else:
    i3_model = load_model(i3_model_file)
else:


    # add a global spatial average pooling layer
    x = base_i3_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    # let's add a fully-connected layer (reduced from 1024 to 124)
    x = Dense(124, activation='relu')(x)
    # and a logistic layer, we have 80 classes
    predictions = Dense(80, activation='softmax')(x)

    i3_model = Model(inputs=base_i3_model.input, outputs=predictions)
    
    if i3_model_json_file:
        with open(i3_model_json_file,'r') as f:
            i3_model = model_from_json(f.read())
        i3_model.load_weights(vgg_model_file)

for layer in base_i3_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
i3_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# serialize model to JSON
model_json = i3_model.to_json()
with open(json_i3, "w") as json_file:
    json_file.write(model_json)

# Resnet Model

In [8]:
base_res_model = ResNet50(include_top=False,
               weights='imagenet')


if os.path.isfile(final_resnet):
  resnet_model = load_model(final_resnet)
elif os.path.isfile(resnet_model_file):
  if os.path.isfile(resnet_model_json_file):
    with open(resnet_model_json_file,'r') as f:
      resnet_model = model_from_json(f.read())
      resnet_model.load_weights(resnet_model_file)
  else:
    resnet_model = load_model(resnet_model_file)
else:

  x = GlobalAveragePooling2D()(base_res_model.output)
  x = Dense(1024, activation='relu')(x)
  x = Dropout(0.3)(x)
  x = Dense(1024, activation='relu')(x)
  prediction = Dense(80, activation='softmax', name='predictions')(x)
  resnet_model = Model(inputs=base_res_model.input, outputs=prediction)
  #resnet_model.summary()




# display(df_resnet[140:])

# warm up model
for layer in base_res_model.layers:
    layer.trainable = False

for i in range(-3,0):
    resnet_model.layers[i].trainable = True
df_resnet = pd.DataFrame(([layer.name, layer.trainable] 
                   for layer in resnet_model.layers), columns=['layer', 'trainable'])
display(df_resnet[-6:])

# compile the model (should be done *after* setting layers to non-trainable)
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# serialize model to JSON
model_json = resnet_model.to_json()
with open(json_resnet, "w") as json_file:
    json_file.write(model_json)


,layer,trainable
174,conv5_block3_out,False
175,global_average_pooling2d_1,True
176,dense_3,True
177,dropout_2,True
178,dense_4,True
179,predictions,True


# Train Models

In [10]:
# Train Model VGG

#Save the model after every epoch.
mc_vgg = ModelCheckpoint(checkpoint_vgg, monitor='val_acc', verbose=1, 
                         save_best_only=True, 
                         save_weights_only=False, mode='auto')

es = EarlyStopping(monitor='val_acc', patience= patience_num,
                   verbose=1, min_delta = 0.0005)


vgg_history = vgg_model.fit_generator(
        vgg_train_generator,
        epochs= epochs,
        validation_data= vgg_validation_generator,
        # steps_per_epoch= vgg_train_samples // batch_size,
        # validation_steps= vgg_validation_samples // batch_size,
        class_weight=class_w,
        callbacks=[mc_vgg, es])

# serialize weights to HDF5
vgg_model.save_weights(weights_vgg)

vgg_model.save(final_vgg)
print("Saved model to disk")

 87/345 [======>.......................] - ETA: 1:03 - loss: 2.4219 - acc: 0.3873
Epoch 00001: val_acc improved from -inf to 0.38730, saving model to competitions/food-recognition-challenge/models/vgg_checkpoint.hdf5
345/345 [==============================] - 288s 835ms/step - loss: 2.6509 - acc: 0.3445 - val_loss: 2.4219 - val_acc: 0.3873
Saved model to disk


In [0]:
# Train InceptionV3 top

#Save the model after every epoch.
mc_i3_top = ModelCheckpoint(checkpoint_top_i3, monitor='val_acc', verbose=1, 
                            save_best_only=True, 
                            save_weights_only=False, mode='auto')

i3_top_history = i3_model.fit_generator(
        i3_train_generator,
        epochs= epochs_top,
        validation_data= vgg_validation_generator,
        steps_per_epoch=i3_train_samples // batch_size,
        validation_steps=i3_validation_samples // batch_size,
        class_weight=class_w,
        callbacks=[mc_i3_top, es])

# serialize weights to HDF5
resnet_model.save_weights(weights_resnet)

In [0]:
# Train InceptionV3 full

#Save the model after every epoch.
mc_i3_full = ModelCheckpoint(checkpoint_full_i3, monitor='val_acc', verbose=1,
                             save_best_only=True, 
                             save_weights_only=False, mode='auto')

# Open up layers
for layer in i3_model.layers[:249]:
    layer.trainable = False
for layer in i3_model.layers[249:]:
    layer.trainable = True

i3_model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
                 loss=categorical_crossentropy, metrics=['accuracy'])

i3_full_history = i3_model.fit_generator(
        i3_train_generator,
        epochs= epochs,
        validation_data= i3_validation_generator,
        steps_per_epoch=i3_train_samples // batch_size,
        validation_steps=i3_validation_samples // batch_size,
        class_weight=class_w,
        callbacks=[mc_i3_top, es])


# serialize weights to HDF5
i3_model.save_weights(weights_i3)

i3_model.save(final_i3)
print("Saved model to disk")

In [0]:
# Train Resnet top

#Save the model after every epoch.
mc_resnet_top = ModelCheckpoint(checkpoint_top_resnet, monitor='val_acc', verbose=1, 
                            save_best_only=True, 
                            save_weights_only=False, mode='auto')

i3_top_history = resnet_model.fit_generator(
        resnet_train_generator,
        epochs=  epochs_top,
        validation_data= resnet_validation_generator,
        steps_per_epoch=resnet_train_samples // batch_size,
        validation_steps=resnet_validation_samples // batch_size,
        class_weight=class_w,
        callbacks=[mc_resnet_top, es])

# serialize weights to HDF5
resnet_model.save_weights(weights_i3)

In [0]:
# Train Resnet full

#Save the model after every epoch.
mc_resnet_full = ModelCheckpoint(checkpoint_full_resnet, monitor='val_acc', verbose=1,
                             save_best_only=True, 
                             save_weights_only=False, mode='auto')

# Open up layers
for layer in resnet_model.layers[:143]:
    layer.trainable = False
for layer in resnet_model.layers[143:]:
    layer.trainable = True

resnet_model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
                 loss=categorical_crossentropy, metrics=['accuracy'])

resnet_full_history = resnet_model.fit_generator(
        resnet_train_generator,
        epochs=  epochs,
        validation_data= resnet_validation_generator,
        steps_per_epoch=resnet_train_samples // batch_size,
        validation_steps=resnet_validation_samples // batch_size,
        class_weight=class_w,
        callbacks=[mc_resnet_full, es])


# serialize weights to HDF5
resnet_model.save_weights(weights_resnet)

resnet_model.save(final_resnet)
print("Saved model to disk")

In [0]:
# Plots VGG16
# Summarize history for accuracy
plt.plot(vgg_history.history['acc'])
plt.plot(vgg_history.history['val_acc'])
plt.title('VGG16 model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.savefig('vgg_acc_1.eps', format='eps', dpi=900)
plt.show()

# Summarize history for loss
plt.plot(vgg_history.history['loss'])
plt.plot(vgg_history.history['val_loss'])
plt.title('VGG16 model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.savefig('vgg_val_1.eps', format='eps', dpi=900)
plt.show()

In [0]:
# Plots Inception V3
# Summarize history for accuracy
plt.plot(i3_top_history.history['acc'])
plt.plot(i3_top_history.history['val_acc'])
plt.title('Inception V3 top model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.tight_layout()
plt.savefig('i3_top_acc_1.eps', format='eps', dpi=900)
plt.show()

# Summarize history for loss
plt.plot(i3_top_history.history['loss'])
plt.plot(i3_top_history.history['val_loss'])
plt.title('Inception V3 top loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.tight_layout()
plt.savefig('i3_top_val_1.eps', format='eps', dpi=900)
plt.show()

# Plots
# Summarize history for accuracy
plt.plot(i3_full_history.history['acc'])
plt.plot(i3_full_history.history['val_acc'])
plt.title('Inception V3 model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.tight_layout()
plt.savefig('i3_acc_1.eps', format='eps', dpi=900)
plt.show()

# Summarize history for loss
plt.plot(i3_full_history.history['loss'])
plt.plot(i3_full_history.history['val_loss'])
plt.title('Inception V3 loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.tight_layout()
plt.savefig('i3_val_1.eps', format='eps', dpi=900)
plt.show()

In [0]:
# Plots Resnet50
# Summarize history for accuracy
plt.plot(resnet_top_history.history['acc'])
plt.plot(resnet_top_history.history['val_acc'])
plt.title('Resnet 50 top model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.tight_layout()
plt.savefig('resnet_top_acc_1.eps', format='eps', dpi=900)
plt.show()

# Summarize history for loss
plt.plot(resnet_top_history.history['loss'])
plt.plot(resnet_top_history.history['val_loss'])
plt.title('Resnet 50  top loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.tight_layout()
plt.savefig('resnet_top_val_1.eps', format='eps', dpi=900)
plt.show()

# Plots
# Summarize history for accuracy
plt.plot(resnet_full_history.history['acc'])
plt.plot(resnet_full_history.history['val_acc'])
plt.title('Resnet 50 model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.tight_layout()
plt.savefig('resnet_acc_1.eps', format='eps', dpi=900)
plt.show()

# Summarize history for loss
plt.plot(resnet_full_history.history['loss'])
plt.plot(resnet_full_history.history['val_loss'])
plt.title('Resnet 50 loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.tight_layout()
plt.savefig('resnet_val_1.eps', format='eps', dpi=900)
plt.show()

# Evaluate models

In [0]:
vgg_evaluate = vgg_model.evaluate_generator(vgg_hold_generator,verbose=1)
print(vgg_evaluate)
print(vgg_model.metrics_names)

i3_evaluate = i3_model.evaluate_generator(i3_hold_generator,verbose=1)
print(i3_evaluate)
resnet_evaluate = resnet_model.evaluate_generator(resnet_hold_generator,verbose=1)
print(i3_evaluate)

total_performance = vgg_evaluate[1] + i3_evaluate[1] + resnet_evaluate[1]
print(total_performance)
weigth_vgg = vgg_evaluate[1]/total_performance
weigth_i3 = i3_evaluate[1]/total_performance
weigth_resnet = resnet_evaluate[1]/total_performance
print(f"""Weight VGG Model: {weigth_vgg}\n Weight Inception V3 Model: {weigth_i3}
  \n Weight Resnet Model: {weigth_resnet}""")

# Predictions

In [0]:
# VGG
# Reset to ensure files are loaded in the correct order
vgg_test_generator.reset()
# Genarate predictions
vgg_predict = vgg_model.predict_generator(vgg_test_generator, verbose = 1)

# Test if amount of predictions match the number of examples
print("Amount of predictions match the number of examples: ",
      len(vgg_predict) == sample_csv.shape[0])

# Inception V3
# Reset to ensure files are loaded in the correct order
i3_test_generator.reset()
# Genarate predictions
i3_predict = i3_model.predict_generator(i3_test_generator, verbose = 1)

# Resnet
# Reset to ensure files are loaded in the correct order
resnet_test_generator.reset()
# Genarate predictions
resnet_predict = resnet_model.predict_generator(resnet_test_generator, verbose = 1)

# Test if amount of predictions match the number of examples
print("Amount of predictions match the number of examples: ",
      len(vgg_predict) ==len(i3_predict) == len(resnet_predict) == sample_csv.shape[0])

print("All indices match: ",
      vgg_mapping_indices == i3_mapping_indices == resnet_mapping_indices)

print("All filenames match: ",
      resnet_test_generator.filenames == resnet_test_generator.filenames == resnet_test_generator.filenames)

# Match highest scoring column index to label
labels = dict((v,k) for k,v in vgg_mapping_indices.items())
# Add labels of both models together
total_predict = (vgg_predict*weigth_vgg  + i3_predict*weigth_i3 + resnet_predict*weigth_resnet)

print(total_predict.shape)
predictions = [labels[k] for k in np.argmax(total_predict,axis=1)]

prediction = pd.DataFrame({"img_name": vgg_test_generator.filenames, "label":predictions})
display(prediction.head())
# Save submission as csv
prediction.to_csv(submission_file, index = False)

# Save files

In [0]:
!ls competitions/food-recognition-challenge/models
!ls gdrive/My\ Drive/Colab\ Notebooks/Food\ Kaggle
!mkdir gdrive/My\ Drive/Colab\ Notebooks/Food\ Kaggle/Model_files
!cp competitions/food-recognition-challenge/models/* gdrive/My\ Drive/Colab\ Notebooks/Food\ Kaggle/Model_files
!ls gdrive/My\ Drive/Colab\ Notebooks/Food\ Kaggle/Model_files
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=False)

Submit

In [0]:
# !kaggle competitions submit -c food-recognition-challenge -f competitions/food-recognition-challenge/submission.csv -m "Submission using Google Colab, model Antonio, 200 epochs"

In [0]:
# !kaggle competitions submissions -c food-recognition-challenge